# Advanced Graph Analyses using the TERRA APIs

This notebook demonstrates how to perform advanced analytical workflows on TERRA data
that are **not directly available through the TERRA web dashboard**.

While the dashboard provides an interactive and exploratory view of international trade
networks, some analyses require:
- customized parameter settings,
- extended time ranges,
- or tailored post-processing steps.

To support these use cases, this notebook accesses TERRA data **directly via the official
TERRA APIs**, using a lightweight Python client (`terra_api`) and standard data
analysis tools.

The focus is on reproducibility and transparency: all data are retrieved programmatically
from the APIs and processed step by step within the notebook.


### Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from terra_modules.terra_api import fetch_graph_time_range, fetch_graph_metrics

## Dataset and analysis parameters

This section defines the main parameters used to query the TERRA Graph API.

The analysis can be performed on different trade network configurations
(e.g. intra-EU or extra-EU trade) and at different temporal frequencies
(monthly or quarterly). Alternative configurations are reported in the cell
below and can be activated by uncommenting the relevant lines.

The base payload specifies the subset of trade flows and attributes to be
included in the graph construction, such as products, means of transport,
and flow direction.

### Reference classifications for graph parameters

Some parameters used in the graph queries (e.g. products and means of transport)
are based on predefined classifications exposed by the TERRA API.

The following endpoints can be used to explore the available categories and
identify valid values to be passed to the `BASE_PAYLOAD`:

- **Products (Extra-EU trade)**  
  https://api.terra.istat.it/cls/productsExtra?lang=en

- **Products (Intra-EU trade)**  
  https://api.terra.istat.it/cls/productsIntra?lang=en

- **Means of transport**  
  https://api.terra.istat.it/cls/transports?lang=en


In [ ]:
# Dataset Graph EU - Extra EU
# dataset = "Extra"

# Dataset Graph EU - World
DATASET = "Intra"

# Monthly data  
FREQUENCY = "month"

# Quarterly data
# FREQUENCY = "quarter"

# Base payload
BASE_PAYLOAD = {
    "percentage": "50",
    "transport": [0, 1, 2, 3, 4, 5, 7, 8, 9], # means of transport
    "product": "TOT",
    "flow": 0, #import = 1, #export = 2
    "weight": True,
    "position": None,
    "edges": None,
    "collapse": True,
}

## Time range selection

The temporal coverage of the analysis is automatically retrieved from the
TERRA metadata service.

By default, the notebook loads the **maximum available time range** for graph
data. Users can optionally restrict the analysis to a narrower time window
by manually editing `start_date` and `end_date`, provided that the selected
interval lies within the available range.


In [ ]:
# Retrieve available time range from the TERRA API
start_date, end_date = fetch_graph_time_range()

# Optional: set a custom time range within the available interval
start_date = "2021-01"
end_date = "2022-12"

start_date, end_date


## Data retrieval from the TERRA Graph API

In this step, graph metrics are retrieved directly from the TERRA Graph API
using the parameters defined above.

The request is executed for each time period in the selected range, and the
resulting metrics are combined into a single tabular dataset, with one row per
country and time period.


## Structure of the graph metrics dataset

The retrieved graph metrics are stored in a tabular dataset with one row per
country and time period.

The main columns are:

- **country**: reporting country
- **period**: reference time period
- **density**: network density
- **degree**: total degree
- **in_degree**: number of incoming connections
- **out_degree**: number of outgoing connections
- **closeness**: closeness centrality
- **betweenness**: betweenness centrality
- **vulnerability**: vulnerability index
- **distinctiveness**: distinctiveness index


In [ ]:
df_metrics = fetch_graph_metrics(
    dataset=DATASET,
    base_payload=BASE_PAYLOAD,
    start_date=start_date,
    end_date=end_date,
    frequency=FREQUENCY,
    verbose=True
)

In [ ]:
df_metrics.head()

## Example analysis: temporal evolution of graph metrics

This section illustrates a simple example of analysis that can be performed
using the retrieved graph metrics.

A selected network indicator is analysed over time for all reporting countries.
The indicator can be easily changed to explore different dimensions of the
trade network (e.g. degree-based or centrality-based measures).


### Country selection

To improve readability, the analysis can be restricted to a selected subset
of countries. This is particularly useful when visualising time series, as
plotting all available countries at once may result in overly crowded figures.

The list below can be customised depending on the analytical focus.


In [ ]:
df_metrics["country"].unique()

In [ ]:
# Select a subset of countries for visualisation
COUNTRIES = [
    "DE", "FR", "IT", "ES", "NL", "PL"
]

df_filtered = df_metrics[df_metrics["country"].isin(COUNTRIES)]


In [ ]:
# Select the graph metric to analyse
# Available indicators include:
# 'density', 'degree', 'in_degree', 'out_degree',
# 'closeness', 'betweenness', 'vulnerability', 'distinctiveness'
METRIC = "degree"

# Pivot the dataset to time x country format
df_pivot = (
    df_filtered
    .pivot(index="period", columns="country", values=METRIC)
    .sort_index()
)

# Plot all time series
ax = df_pivot.plot(
    figsize=(12, 6),
    linewidth=1,
    legend=False
)

ax.set_title(f"Evolution of {METRIC.replace('_', ' ').title()}")
ax.set_xlabel("Time")
ax.set_ylabel(METRIC.replace("_", " ").title())
ax.grid(True, alpha=0.3)

plt.legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()


## Focus on a single country

In this final example, the temporal evolution of a selected graph metric is
analysed for a single country. This type of analysis is useful to investigate
country-specific dynamics that may be less visible in multi-country plots.


In [ ]:
# Select country and metric
COUNTRY = "DE"

# Select the graph metric to analyse
# Available indicators include:
# 'density', 'degree', 'in_degree', 'out_degree',
# 'closeness', 'betweenness', 'vulnerability', 'distinctiveness'
METRIC = "degree"

# Filter data for the selected country
df_country_metrics = (
    df_metrics[df_metrics["country"] == COUNTRY]
    .sort_values("period")
)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(
    df_country_metrics["period"],
    df_country_metrics[METRIC],
    linewidth=2
)

plt.title(
    f"{COUNTRY} – {METRIC.replace('_', ' ').title()} over time"
)
plt.xlabel("Time")
plt.xticks(rotation=45)

plt.ylabel(METRIC.replace("_", " ").title())
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
